# Setup

In [ ]:
!pip install yt-dlp

from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Manager
from display import replace_watch_comment
from getChat import get_chat
from IPython.display import clear_output
from datetime import datetime, timedelta, timezone


In [ ]:
import getpass
url = input("YoutubeのURLを入力してください: ")
api_key = getpass.getpass("chatGPTを利用する方は、API keyを自己責任で入力してください。: ")
split_url = url.split('https://www.youtube.com/watch?v=')
assert split_url[0] == ''
live_id = split_url[1]

# Promptの設定

In [ ]:
prompt_setting = {
    'model': "gpt-3.5-turbo",
    'messages': [
        {
            "role": "system",
            "content": "あなたはコメント分析の専門家です"
        },
        {
            "role": "user",
            "content": """
                次の一連のコメントから、コメント主の特徴を一言で分析してください。
                「昨日はラーメン食べた」
                「今日はパスタ食べた」
                「明日はうどん食う」
            """
        },
        {
            "role": "assistant",
            "content": "麺好き"
        },
        {"role": "user",
            "content": """
                素晴らしい。
                次の一連のコメントから、コメント主の特徴を一言で分析してください。
                「昨日カレー食べた」
                「今日はカレーグラタン食べた」
                「明日はカレーうどん食う」
            """
        },
        {
            "role": "assistant",
            "content": "カレー好き"
        },
        {
            "role": "user",
            "content":  (
                "素晴らしい。次の一連のコメントから、コメント主の特徴を一言で分析してください。"
            )
        },
    ],
    'temperature': 1 
}


# 中略

In [ ]:
def main(live_id:str):
    with Manager() as manager:
        queue = manager.Queue()
        with ProcessPoolExecutor(max_workers=2) as executor:
            future_1 = executor.submit(replace_watch_comment, queue, live_id, api_key,prompt_setting)
            future_2 = executor.submit(get_chat, live_id)
            while True:
                message = queue.get()
                if message != '':
                    clear_output()
                    print(message)
                elif message == 'DONE':
                    break
                    
            future_1.result()
            future_2.result()



# 下記に、チャットとその分析結果が表示されます
## 初回のチャットが分析されるまで、数十秒かかる場合があります

In [ ]:
main(live_id)